# DualGAN

ref. YI, Zili et al.  
     Dualgan: Unsupervised dual learning for image-to-image translation.  
     In: Proceedings of the IEEE international conference on computer vision. 2017. p. 2849-2857.
     
![Dual GAN architecture](arch.png)

In [1]:
import os
import numpy as np
import scipy
import matplotlib.pyplot as plt

In [2]:
from keras.datasets import mnist
from keras.layers import Input, Dense, Dropout
from keras.layers import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
from keras.optimizers import Adam
import keras.backend as K

Using TensorFlow backend.


## Generator

In [3]:
def build_generator(img_dim):

    X = Input(shape=(img_dim,))

    model = Sequential()
    model.add(Dense(256, input_dim=img_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dropout(0.4))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dropout(0.4))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dropout(0.4))
    model.add(Dense(img_dim, activation='tanh'))

    X_translated = model(X)

    return Model(X, X_translated)

## Discriminator

In [4]:
def build_discriminator(img_dim):

    img = Input(shape=(img_dim,))

    model = Sequential()
    model.add(Dense(512, input_dim=img_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(1))

    validity = model(img)

    return Model(img, validity)

# Helper

In [5]:
def sample_generator_input(X, batch_size):
    # Sample random batch of images from X
    idx = np.random.randint(0, X.shape[0], batch_size)
    return X[idx]

In [6]:
def wasserstein_loss(y_true, y_pred):
    return K.mean(y_true * y_pred)

In [7]:
def save_imgs(G_AB, G_BA, epoch, 
              X_A, X_B, 
              img_rows, img_cols):
    r, c = 4, 4

    # Sample generator inputs
    imgs_A = sample_generator_input(X_A, c)
    imgs_B = sample_generator_input(X_B, c)

    # Images translated to their opposite domain
    fake_B = G_AB.predict(imgs_A)
    fake_A = G_BA.predict(imgs_B)

    gen_imgs = np.concatenate([imgs_A, fake_B, imgs_B, fake_A])
    gen_imgs = gen_imgs.reshape((r, c, img_rows, img_cols, 1))

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[i, j, :,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("images/mnist_%d.png" % epoch)
    plt.close()

In [8]:
def train(D_A, D_B, G_AB, G_BA, combined,
          img_dim, img_rows, img_cols,
          epochs, batch_size=128, sample_interval=50):

    # Load the dataset
    (X_train, _), (_, _) = mnist.load_data()

    # Rescale -1 to 1
    X_train = (X_train.astype(np.float32) - 127.5) / 127.5

    # Domain A and B (rotated)
    X_A = X_train[:int(X_train.shape[0]/2)]
    X_B = scipy.ndimage.interpolation.rotate(X_train[int(X_train.shape[0]/2):], 90, axes=(1, 2))

    X_A = X_A.reshape(X_A.shape[0], img_dim)
    X_B = X_B.reshape(X_B.shape[0], img_dim)

    clip_value = 0.01
    n_critic = 4

    # Adversarial ground truths
    valid = -np.ones((batch_size, 1))
    fake = np.ones((batch_size, 1))

    for epoch in range(epochs):

        # Train the discriminator for n_critic iterations
        for _ in range(n_critic):

            # ----------------------
            #  Train Discriminators
            # ----------------------

            # Sample generator inputs
            imgs_A = sample_generator_input(X_A, batch_size)
            imgs_B = sample_generator_input(X_B, batch_size)

            # Translate images to their opposite domain
            fake_B = G_AB.predict(imgs_A)
            fake_A = G_BA.predict(imgs_B)

            # Train the discriminators
            D_A_loss_real = D_A.train_on_batch(imgs_A, valid)
            D_A_loss_fake = D_A.train_on_batch(fake_A, fake)

            D_B_loss_real = D_B.train_on_batch(imgs_B, valid)
            D_B_loss_fake = D_B.train_on_batch(fake_B, fake)

            D_A_loss = 0.5 * np.add(D_A_loss_real, D_A_loss_fake)
            D_B_loss = 0.5 * np.add(D_B_loss_real, D_B_loss_fake)

            # Clip discriminator weights
            for d in [D_A, D_B]:
                for l in d.layers:
                    weights = l.get_weights()
                    weights = [np.clip(w, -clip_value, clip_value) for w in weights]
                    l.set_weights(weights)

        # ------------------
        #  Train Generators
        # ------------------

        # Train the generators
        g_loss = combined.train_on_batch([imgs_A, imgs_B], [valid, valid, imgs_A, imgs_B])

        # Plot the progress
        print ("%d [D1 loss: %f] [D2 loss: %f] [G loss: %f]" \
            % (epoch, D_A_loss[0], D_B_loss[0], g_loss[0]))

        # If at save interval => save generated image samples
        if epoch % sample_interval == 0:
            save_imgs(G_AB, G_BA, epoch, 
                      X_A, X_B, 
                      img_rows, img_cols)

## main()

In [ ]:
if not os.path.exists('images'):
    os.makedirs('images')

In [9]:
img_rows = 28
img_cols = 28
channels = 1
img_dim = img_rows * img_cols

In [10]:
# create optimizer
optimizer = Adam(0.0002, 0.5)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [11]:
# Build and compile the discriminators
D_A = build_discriminator(img_dim)
D_A.compile(loss=wasserstein_loss,
            optimizer=optimizer,
            metrics=['accuracy'])
D_B = build_discriminator(img_dim)
D_B.compile(loss=wasserstein_loss,
            optimizer=optimizer,
            metrics=['accuracy'])

In [12]:
#-------------------------
# Construct Computational
#   Graph of Generators
#-------------------------

In [13]:
# Build the generators
G_AB = build_generator(img_dim)
G_BA = build_generator(img_dim)

In [14]:
# For the combined model we will only train the generators
D_A.trainable = False
D_B.trainable = False

In [15]:
# The generator takes images from their respective domains as inputs
imgs_A = Input(shape=(img_dim,))
imgs_B = Input(shape=(img_dim,))

In [16]:
# Generators translates the images to the opposite domain
fake_B = G_AB(imgs_A)
fake_A = G_BA(imgs_B)

In [17]:
# The discriminators determines validity of translated images
valid_A = D_A(fake_A)
valid_B = D_B(fake_B)

In [18]:
# Generators translate the images back to their original domain
recov_A = G_BA(fake_B)
recov_B = G_AB(fake_A)

In [19]:
# The combined model  (stacked generators and discriminators)
combined = Model(inputs=[imgs_A, imgs_B], outputs=[valid_A, valid_B, recov_A, recov_B])
combined.compile(loss=[wasserstein_loss, wasserstein_loss, 'mae', 'mae'],
                 optimizer=optimizer,
                 loss_weights=[1, 1, 100, 100])

In [ ]:
# epochs=30000
epochs=5000
train(D_A, D_B, G_AB, G_BA, combined,
      img_dim, img_rows, img_cols,
      epochs=epochs, batch_size=32, sample_interval=200)

c:\users\henri\appdata\local\programs\python\python37\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
c:\users\henri\appdata\local\programs\python\python37\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
c:\users\henri\appdata\local\programs\python\python37\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D1 loss: 0.000081] [D2 loss: 0.000089] [G loss: 195.662094]


c:\users\henri\appdata\local\programs\python\python37\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1 [D1 loss: 0.000091] [D2 loss: 0.000095] [G loss: 193.996979]
2 [D1 loss: 0.000093] [D2 loss: 0.000096] [G loss: 193.581238]
3 [D1 loss: 0.000093] [D2 loss: 0.000095] [G loss: 193.942459]
4 [D1 loss: 0.000090] [D2 loss: 0.000091] [G loss: 192.923416]
5 [D1 loss: 0.000086] [D2 loss: 0.000086] [G loss: 192.663849]
6 [D1 loss: 0.000082] [D2 loss: 0.000081] [G loss: 192.518951]
7 [D1 loss: 0.000078] [D2 loss: 0.000076] [G loss: 193.187866]
8 [D1 loss: 0.000074] [D2 loss: 0.000071] [G loss: 192.921753]
9 [D1 loss: 0.000070] [D2 loss: 0.000067] [G loss: 191.755005]
10 [D1 loss: 0.000066] [D2 loss: 0.000063] [G loss: 191.921646]
11 [D1 loss: 0.000063] [D2 loss: 0.000060] [G loss: 191.658981]
12 [D1 loss: 0.000060] [D2 loss: 0.000057] [G loss: 191.580139]
13 [D1 loss: 0.000057] [D2 loss: 0.000055] [G loss: 190.972107]
14 [D1 loss: 0.000055] [D2 loss: 0.000053] [G loss: 190.847275]
15 [D1 loss: 0.000053] [D2 loss: 0.000051] [G loss: 189.069412]
16 [D1 loss: 0.000051] [D2 loss: 0.000050] [G los

130 [D1 loss: 0.000039] [D2 loss: 0.000039] [G loss: 80.510185]
131 [D1 loss: 0.000039] [D2 loss: 0.000039] [G loss: 89.073471]
132 [D1 loss: 0.000039] [D2 loss: 0.000039] [G loss: 89.895767]
133 [D1 loss: 0.000039] [D2 loss: 0.000039] [G loss: 81.892517]
134 [D1 loss: 0.000039] [D2 loss: 0.000039] [G loss: 78.199097]
135 [D1 loss: 0.000039] [D2 loss: 0.000039] [G loss: 76.666214]
136 [D1 loss: 0.000039] [D2 loss: 0.000039] [G loss: 72.417206]
137 [D1 loss: 0.000039] [D2 loss: 0.000039] [G loss: 84.458305]
138 [D1 loss: 0.000039] [D2 loss: 0.000039] [G loss: 78.730972]
139 [D1 loss: 0.000039] [D2 loss: 0.000039] [G loss: 77.482674]
140 [D1 loss: 0.000039] [D2 loss: 0.000039] [G loss: 78.629974]
141 [D1 loss: 0.000039] [D2 loss: 0.000039] [G loss: 76.981247]
142 [D1 loss: 0.000039] [D2 loss: 0.000039] [G loss: 77.052887]
143 [D1 loss: 0.000039] [D2 loss: 0.000039] [G loss: 81.799057]
144 [D1 loss: 0.000038] [D2 loss: 0.000038] [G loss: 78.787079]
145 [D1 loss: 0.000038] [D2 loss: 0.0000

...

1159 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 50.317703]
1160 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 43.137169]
1161 [D1 loss: 0.000035] [D2 loss: 0.000034] [G loss: 49.479607]
1162 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 44.935310]
1163 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 43.015831]
1164 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 46.116837]
1165 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 48.371574]
1166 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 44.414032]
1167 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 44.190308]
1168 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 46.419037]
1169 [D1 loss: 0.000034] [D2 loss: 0.000035] [G loss: 47.101730]
1170 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 49.756294]
1171 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 47.521317]
1172 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 43.139534]
1173 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 49.926872]
1174 [D1 loss: 0.000034] 

1286 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 47.499760]
1287 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 45.083450]
1288 [D1 loss: 0.000035] [D2 loss: 0.000034] [G loss: 50.276840]
1289 [D1 loss: 0.000035] [D2 loss: 0.000034] [G loss: 44.485619]
1290 [D1 loss: 0.000035] [D2 loss: 0.000034] [G loss: 45.410225]
1291 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 47.610054]
1292 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 42.359489]
1293 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 43.603546]
1294 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 48.760635]
1295 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 45.911972]
1296 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 46.548923]
1297 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 45.011684]
1298 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 46.395035]
1299 [D1 loss: 0.000034] [D2 loss: 0.000035] [G loss: 42.209057]
1300 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 47.568718]
1301 [D1 loss: 0.000034] 

1413 [D1 loss: 0.000035] [D2 loss: 0.000034] [G loss: 44.166367]
1414 [D1 loss: 0.000034] [D2 loss: 0.000035] [G loss: 46.228210]
1415 [D1 loss: 0.000034] [D2 loss: 0.000035] [G loss: 45.789387]
1416 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 47.997589]
1417 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 40.618301]
1418 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 42.659981]
1419 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 48.297180]
1420 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 45.759132]
1421 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 44.102978]
1422 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 45.106361]
1423 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 47.641457]
1424 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 44.325310]
1425 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 51.167019]
1426 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 46.267841]
1427 [D1 loss: 0.000035] [D2 loss: 0.000034] [G loss: 45.213257]
1428 [D1 loss: 0.000034] 

1540 [D1 loss: 0.000034] [D2 loss: 0.000035] [G loss: 45.829750]
1541 [D1 loss: 0.000034] [D2 loss: 0.000035] [G loss: 42.549973]
1542 [D1 loss: 0.000034] [D2 loss: 0.000035] [G loss: 44.168243]
1543 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 43.401176]
1544 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 43.824863]
1545 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 48.368816]
1546 [D1 loss: 0.000034] [D2 loss: 0.000035] [G loss: 40.621487]
1547 [D1 loss: 0.000034] [D2 loss: 0.000035] [G loss: 47.795319]
1548 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 43.461281]
1549 [D1 loss: 0.000035] [D2 loss: 0.000034] [G loss: 42.265648]
1550 [D1 loss: 0.000035] [D2 loss: 0.000034] [G loss: 44.586288]
1551 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 46.946854]
1552 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 46.251400]
1553 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 42.507256]
1554 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 40.561157]
1555 [D1 loss: 0.000034] 

1667 [D1 loss: 0.000035] [D2 loss: 0.000034] [G loss: 43.559555]
1668 [D1 loss: 0.000035] [D2 loss: 0.000034] [G loss: 42.197517]
1669 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 40.391106]
1670 [D1 loss: 0.000034] [D2 loss: 0.000035] [G loss: 41.502930]
1671 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 40.996590]
1672 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 50.241615]
1673 [D1 loss: 0.000034] [D2 loss: 0.000035] [G loss: 42.945618]
1674 [D1 loss: 0.000034] [D2 loss: 0.000035] [G loss: 45.588741]
1675 [D1 loss: 0.000034] [D2 loss: 0.000035] [G loss: 41.069603]
1676 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 41.793938]
1677 [D1 loss: 0.000035] [D2 loss: 0.000034] [G loss: 41.985825]
1678 [D1 loss: 0.000035] [D2 loss: 0.000034] [G loss: 45.848518]
1679 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 41.428421]
1680 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 46.836441]
1681 [D1 loss: 0.000034] [D2 loss: 0.000035] [G loss: 44.780403]
1682 [D1 loss: 0.000034] 